In [ ]:
import pandas as pd
from datetime import datetime
from googleapiclient.discovery import build

# Proceso y Metodología de Extracción de Comentarios

Se busca extraer los links de los videos de los canales seleccionados en el dataset de **noticias** hasta que estos sumen, como mínimo, 250,000 comentarios en su conjunto, para aquellos que tienen una categoría distinta a "Internacional".

## Criterio de Selección

- **Palabras Claves**: Las palabras claves estarán contenidas en el dataset 'claves' (nombre del presidente, país del presidente) junto con la palabra "presidente".
- **Número mínimo de comentarios:** El minimo de número de comentarios del video son 100 para que este sea considerado en la lista.
- **Dataset 'claves'**: Este dataset tiene 2 columnas: una llamada "País" y otra "Nombre del Presidente". Para esta búsqueda solo se usará la columna "Nombre del Presidente".
- **Periodo de Tiempo**: Los videos deben ser del último año.
- **Filtrado Inicial**: Inicialmente, solo se buscará filtrar los videos que contengan los criterios de las palabras y conocer cuántos comentarios tienen.

In [ ]:
noticias=  pd.read_excel('canales_noticias_con_id.xlsx')

In [ ]:
noticias.head()

In [ ]:
claves = pd.read_excel("data/tables/presidentes2024.xlsx")

In [ ]:
claves.head()

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

### Descripción de la Función `get_video_details`

#### Propósito
Obtiene detalles de un video en YouTube utilizando la API de YouTube Data.

#### Parámetro
- `video_id`: El identificador único del video en YouTube.

#### Funcionamiento
1. Llama a la API de YouTube para obtener datos del video (`snippet`, `contentDetails`, `statistics`).
2. Verifica si la respuesta contiene elementos y selecciona el primer video.
3. Extrae y almacena los siguientes detalles:
   - Título del video
   - Descripción
   - Fecha de publicación
   - Duración
   - Número de comentarios
4. Crea un diccionario con estos detalles y lo devuelve. Si no hay datos, devuelve `None`.

In [ ]:
def get_video_details(video_id):
    video_response = youtube.videos().list(
        part="snippet,contentDetails,statistics", # se recolecta la información de los metadatos del video y las estadísticas
        id=video_id
    ).execute()
    
    if "items" in video_response and len(video_response["items"]) > 0: # si se encontró el video en youtube se extrae la información
        video = video_response["items"][0] # se extrae la información del video en cuestión 
        title = video["snippet"]["title"] # se extrae el título del video
        description = video["snippet"]["description"] # se extrae la descripción del video
        published_at = video["snippet"]["publishedAt"] # se extrae la fecha de publicación del video
        duration = video["contentDetails"]["duration"] # se extrae la duración del video
        comment_count = int(video["statistics"].get("commentCount", 0)) # se extrae el número de comentarios del video 
        video_data = { # se crea un diccionario con la información del video
            "Nombre del Video": title,
            "Link del Video": f"https://www.youtube.com/watch?v={video_id}",
            "Numero de Comentarios": comment_count,
            "Duración del Video": duration,
            "Fecha del Video": published_at
        }
        return video_data
    return None

In [ ]:
# Concatenar las columnas 'País' y 'Nombre del Presidente' de 'claves' y añadir la palabra "presidente"
claves['Query'] = claves['País'] + " " + claves['Nombre del Presidente'] + " presidente"

## Extracción de datos de enero

In [ ]:
# Obtener la fecha actual y establecer las fechas de inicio y fin para el mes de enero del año actual
current_date = datetime.now()
january_first = datetime(current_date.year, 1, 1).isoformat("T") + "Z"
january_end = datetime(current_date.year, 1, 31, 23, 59, 59).isoformat("T") + "Z"

### Función para buscar videos en un canal con palabras clave específicas y dentro del rango de fechas especificado

#### Propósito
Busca videos en un canal de YouTube que coincidan con palabras clave específicas y un rango de fechas determinado.

#### Parámetros
- `channel_id`: El identificador único del canal de YouTube.
- `query`: Las palabras clave para buscar videos.
- `published_after`: La fecha de inicio del rango (en formato ISO 8601).
- `published_before`: La fecha de fin del rango (en formato ISO 8601).
- `max_results` (opcional): El número máximo de resultados a devolver (por defecto, 5).

#### Funcionamiento
1. **Llamada a la API de Búsqueda de YouTube**:
   - Realiza una búsqueda en el canal especificado utilizando las palabras clave y el rango de fechas.
   - Ordena los resultados por cantidad de visualizaciones y limita el número de resultados según `max_results`.

2. **Recopilación de Resultados**:
   - Itera sobre los resultados de búsqueda.
   - Para cada video encontrado, obtiene detalles específicos utilizando la función `get_video_details`.

3. **Filtrado y Almacenamiento de Videos**:
   - Filtra los videos que tienen más de 100 comentarios.
   - Almacena los datos de los videos que cumplen con este criterio en una lista.

4. **Devolución de Resultados**:
   - Devuelve la lista de videos que cumplen con los criterios de búsqueda y tienen más de 100 comentarios.

In [ ]:
# Función para buscar videos en un canal con palabras clave específicas y dentro del rango de fechas especificado
def search_videos(channel_id, query, published_after, published_before, max_results=5):
    search_response = youtube.search().list(
        channelId=channel_id, # se busca en el canal con el ID proporcionado
        q=query, # se busca la palabra clave en los títulos y descripciones
        type="video", # se buscan solo videos
        part="id,snippet", # se recolectan los metadatos del video y la información de la búsqueda
        maxResults=max_results, # se obtienen 5 resultados como máximo por búsqueda dado el límite de cuotas
        order='viewCount', # se ordenan los resultados por vistas
        publishedAfter=published_after, # se obtienen solo los videos publicados después de la fecha especificada
        publishedBefore=published_before   # se obtienen solo los videos publicados antes de la fecha especificada
    ).execute()
    
    videos = []
    for search_result in search_response.get("items", []): # se recorren los resultados de la búsqueda
        video_id = search_result["id"]["videoId"] # se extrae el ID del video
        video_data = get_video_details(video_id) # se obtiene la información detallada del video
        if video_data and video_data["Numero de Comentarios"] > 100: # si se encontró información y el video tiene más de 100 comentarios
            videos.append(video_data) # se añade la información del video a la lista de videos
    return videos


In [ ]:
noticias

In [ ]:
claves

### Función para buscar videos en múltiples canales y guardarlos en un archivo CSV

#### Propósito
Busca videos en múltiples canales de YouTube utilizando palabras clave específicas dentro de un rango de fechas y guarda los resultados en un archivo CSV.

#### Parámetros
- `noticias`: DataFrame que contiene información sobre los noticieros, incluidas las categorías, nombres y IDs de los canales.
- `claves`: DataFrame que contiene las palabras clave para la búsqueda, incluyendo los nombres de los presidentes y sus países.
- `january_first`: Fecha de inicio del rango (1 de enero en formato ISO 8601).
- `january_end`: Fecha de fin del rango (31 de enero en formato ISO 8601).

#### Funcionamiento
1. **Inicialización**:
   - Crea una lista vacía `videos` para almacenar los datos de los videos encontrados.

2. **Iteración sobre los canales**:
   - Recorre cada fila del DataFrame `noticias`.
   - Extrae el `ChannelID`, `Categoría` y `Nombre del noticiero`.

3. **Verificación y Búsqueda de Videos**:
   - Si el `ChannelID` no es nulo, itera sobre cada fila del DataFrame `claves`.
   - Extrae la `Query` (consulta de búsqueda) y el `País`.
   - Llama a la función `search_videos` con el `channel_id`, `query`, `january_first` y `january_end`.

4. **Procesamiento de Resultados de Búsqueda**:
   - Para cada resultado de búsqueda, si el video tiene más de 100 comentarios, se agregan datos adicionales:
     - `Nombre del noticiero`
     - `Query` utilizada
     - `País` (se ajusta dependiendo de si la categoría es "Internacional")

5. **Almacenamiento de Resultados**:
   - Los detalles de los videos encontrados se agregan a la lista `videos`.

6. **Guardado en CSV**:
   - Se convierte la lista `videos` en un DataFrame `df_videos`.
   - Se guarda el DataFrame en un archivo CSV (`./videos-per-month/enero.csv`).

7. **Actualización de `claves`**:
   - Se concatena `País` y `Nombre del Presidente` en `claves` para formar la `Query`.

In [ ]:
# Buscar videos y guardar en CSV
videos = []
for _, row in noticias.iterrows(): # Recorrer cada fila de 'noticias'
    channel_id = row['ChannelID'] # Obtener el ID del canal de la fila actual
    categoria = row['Categoría'] # Obtener la categoría del noticiero
    nombre_noticiero = row['Nombre del noticiero'] # Obtener el nombre del noticiero
    
    if pd.notnull(channel_id): # Si el ID del canal no es nulo
        for _, clave_row in claves.iterrows(): # Recorrer cada fila de 'claves'
            query = clave_row['Query'] # Obtener la consulta de la fila actual
            country = clave_row['País'] # Obtener el país de la fila actual
            search_results = search_videos(channel_id, query, january_first, january_end) # Buscar videos en el canal con la consulta y fechas especificadas
            for video in search_results: # Recorrer cada video encontrado en los resultados de la búsqueda
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero # Agregar el nombre del noticiero a los datos del video
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional": # Si la categoría del noticiero es internacional
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria + " " + country # Agregar el país a los datos del video
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/enero.csv", index=False)

print("Videos data saved to ./videos-per-month/enero.csv")

In [ ]:
df_videos["Numero de Comentarios"].sum()

## Extracción de datos de febrero

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
current_date = datetime.now()
february_first = datetime(current_date.year, 2, 1).isoformat("T") + "Z"
february_end = datetime(current_date.year, 2, 29, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero.

In [ ]:
# Buscar videos y guardar en CSV
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, february_first, february_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/febrero.csv", index=False)

print("Videos data saved to ./videos-per-month/febrero.csv")

In [ ]:
df_videos["Numero de Comentarios"].sum()

## Extracción de datos de Marzo

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
current_date = datetime.now()
march_first = datetime(current_date.year, 3, 1).isoformat("T") + "Z"
march_end = datetime(current_date.year, 3, 31, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero.

In [ ]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, march_first, march_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/marzo.csv", index=False)

print("Videos data saved to ./videos-per-month/marzo.csv")

In [ ]:
df_videos["Numero de Comentarios"].sum()

## Extracción de datos de Abril

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
current_date = datetime.now()
april_first = datetime(current_date.year, 4, 1).isoformat("T") + "Z"
april_end = datetime(current_date.year, 4, 30, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [ ]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, april_first, april_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/abril.csv", index=False)

print("Videos data saved to ./videos-per-month/abril.csv")

In [ ]:
df_videos["Numero de Comentarios"].sum()

## Extracción de datos de Mayo

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
current_date = datetime.now()
may_first = datetime(current_date.year, 5, 1).isoformat("T") + "Z"
may_end = datetime(current_date.year, 5, 31, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [ ]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, may_first, may_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/mayo2.csv", index=False)

print("Videos data saved to ./videos-per-month/mayo.csv")

In [ ]:
df_videos["Numero de Comentarios"].sum()

## Extracción de datos de Junio

In [ ]:
api_key = "your_api_key"

In [ ]:
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
current_date = datetime.now()
june_first = datetime(current_date.year, 6, 1).isoformat("T") + "Z"
june_end = datetime(current_date.year, 6, 30, 23, 59, 59).isoformat("T") + "Z"

Se vuelve a correr la consulta de el mes de enero con las fechas modificadas.

In [ ]:
videos = []
for _, row in noticias.iterrows():
    channel_id = row['ChannelID']
    categoria = row['Categoría']
    nombre_noticiero = row['Nombre del noticiero']
    
    if pd.notnull(channel_id):
        for _, clave_row in claves.iterrows():
            query = clave_row['Query']
            country = clave_row['País']
            search_results = search_videos(channel_id, query, june_first, june_end)
            for video in search_results:
                if video:
                    video["Nombre del noticiero"] = nombre_noticiero
                    video["Query"] = query  # Agregar la consulta a los datos del video
                    if categoria == "Internacional":
                        video["País"] = f"Internacional + {country}"
                    else:
                        video["País"] = categoria
                    videos.append(video)

# Crear DataFrame y guardar en CSV
df_videos = pd.DataFrame(videos)
df_videos.to_csv("./videos-per-month/junio.csv", index=False)

print("Videos data saved to ./videos-per-month/junio.csv")

junio = pd.rea

In [ ]:
junio = pd.read_csv("videos-per-month/junio.csv")

In [ ]:
juniocomments = junio["Numero de Comentarios"].sum()

juniocomments

# General Analysis

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# Sumar el número de comentarios de todos los videos
# leer los archivos csv
enero = pd.read_csv("./videos-per-month/enero.csv")
febrero = pd.read_csv("./videos-per-month/febrero.csv")
marzo = pd.read_csv("./videos-per-month/marzo.csv")
abril = pd.read_csv("./videos-per-month/abril.csv")
mayo = pd.read_csv("./videos-per-month/mayo.csv")
junio = pd.read_csv("videos-per-month/junio.csv")

# Concatenar los DataFrames
anual = pd.concat([enero, febrero, marzo, abril, mayo, junio])

# sumar el número de comentarios
total_comments = anual["Numero de Comentarios"].sum()


Se une en un solo dataset todos los meses extraidos.

In [ ]:
completo = anual

In [ ]:
completo["Numero de Comentarios"].sum() # completo = 

In [ ]:
# necesito la grfica segmentada por pais

# Crear una columna para diferenciar entre Internacional y Nacional

completo['Tipo'] = completo['País'].apply(lambda x: 'Internacional' if 'Internacional' in x else 'Nacional')

# Agrupar los datos por 'País' y 'Tipo', sumando el número de comentarios
grouped_data = completo.groupby(['País', 'Tipo'])['Numero de Comentarios'].sum().unstack().fillna(0).reset_index()

# Crear la gráfica de barras apiladas
fig = go.Figure(data=[
    go.Bar(name='Internacional', x=grouped_data['País'], y=grouped_data['Internacional'], marker_color='blue'),
    go.Bar(name='Nacional', x=grouped_data['País'], y=grouped_data['Nacional'], marker_color='red')
])

# Cambiar el diseño de la gráfica para apilar las barras

fig.update_layout(barmode='stack', title='Comparación del Número de Comentarios por País',
                    xaxis_title='País', yaxis_title='Número de Comentarios')

# Mostrar la
fig.show()

In [ ]:
completo.to_csv("completo.csv", index=False) # se renomnro a com_anuales para evitar confusiones con el nombre del archivo

In [ ]:
completo 

In [ ]:
completo["Numero de Comentarios"].sum()

In [ ]:
completo

# Identificando Duplicados y presidentes relacionados

In [ ]:
import pandas as pd

In [ ]:
com_anuales = pd.read_csv("./com_anuales.csv")

In [ ]:
com_anuales["Numero de Comentarios"].sum()

In [ ]:
# mostrar duplicados

com_anuales[com_anuales.duplicated(subset="Link del Video", keep=False)]

In [ ]:
# mostrar filas con el mismo link en general Y NOMBRE para identificar si son duplicados o no

com_anuales[com_anuales.duplicated(subset="Link del Video", keep=False) & com_anuales.duplicated(subset="Nombre del Video", keep=False)]

In [ ]:
# drop completos con el mismo link 
com_anuales = com_anuales.drop_duplicates(subset=['Link del Video'])

com_anuales["Numero de Comentarios"].sum()

In [ ]:
com_anuales.to_csv("com_anuales_nd.csv", index=False) # se almacena el archivo sin duplicados eb com_anuales_nd para la extraccion de comentarios

In [ ]:
# graficar la distribucion de comentarios de com_anuales en el tiempo

import plotly.express as px

com_anuales["Fecha del Video"] = pd.to_datetime(com_anuales["Fecha del Video"])

fig = px.histogram(com_anuales, x="Fecha del Video", title="Distribución de Fechas de Videos en com_anuales")

fig.show()

In [ ]:
com_anuales

In [ ]:
# quitar la palabra internacional y el simbolo + de los valores de la columna pais para estructurar la informacion de manera correcta

com_anuales["País"] = com_anuales["País"].str.replace("Internacional", "").str.replace("+", "").str.strip()

In [ ]:
com_anuales["País"].unique()

In [ ]:
com_anuales.groupby(["Tipo", "País"])["Numero de Comentarios"].sum()


In [ ]:
com_anuales["Numero de Comentarios"].sum()

# Extracción de comentarios

In [ ]:
com_anuales.head(10)

In [ ]:
com_anuales['ID del Video'] = com_anuales['Link del Video'].apply(lambda x: x.split('v=')[-1]) # se obtiene el ID del video de la URL de cada video

In [ ]:
com_anuales

In [ ]:
from googleapiclient.discovery import build
import pandas as pd

In [ ]:
api_key = "your_api_key"
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
def get_video_comments(video_id):
    comments_data = []
    request = youtube.commentThreads().list(
        part='snippet,replies', # se obtienen los metadatos de los comentarios
        videoId=video_id, # se especifica el ID del video
        maxResults=100 # se obtienen 100 comentarios por solicitud
    )
    response = request.execute()

    while request is not None: # se recorren los comentarios de los videos y sus respuestas asociadas si las hay 
        for item in response['items']: # procesamiento de comentarios princiaples
            comment = item['snippet']['topLevelComment']['snippet'] # se extrae la información del comentario
            comment_data = {
                'video_id': video_id, # se almacen el ID del video
                'comment_id': item['id'], # se almacen el ID del comentario
                'text': comment['textDisplay'], # se almacena el texto del comentario
                'likes': comment['likeCount'], # se almacena el número de likes del comentario
                'type': 'principal' # se almacena el tipo de comentario
            }
            comments_data.append(comment_data)
            
            if 'replies' in item: # procesamiento de respuestas a comentarios
                for reply in item['replies']['comments']: # se recorren las respuestas a los comentarios
                    reply_data = {
                        'video_id': video_id, # se almacena el ID del video
                        'comment_id': reply['id'], # se almacena el ID del comentario
                        'text': reply['snippet']['textDisplay'], # se almacena el texto del comentario
                        'likes': reply['snippet']['likeCount'], # se almacena el número de likes del comentario
                        'type': 'reply' # se almacen el tipo de comentario
                    }
                    comments_data.append(reply_data)
        
        if 'nextPageToken' in response: # como los comentarios pueden ser muchos, se hace paginacion, para obtener los siguientes comentarios se usa el token de la siguiente pagina
            request = youtube.commentThreads().list( # se realiza la solicitud para obtener los siguientes comentarios
                part='snippet,replies', # se obtienen los metadatos de los comentarios
                videoId=video_id, # se especifica el ID del video
                pageToken=response['nextPageToken'], # se especifica el token de la siguiente página de comentarios a obtener 
                maxResults=100 # se obtienen 100 comentarios por solicitud
            )
            response = request.execute()
        else:
            request = None
    return comments_data


In [ ]:
all_comments_df = pd.DataFrame(columns=['video_id', 'comment_id', 'text', 'likes', 'type']) # se crea un DataFrame vacío para almacenar los comentarios

# Iterar sobre cada ID de video y extraer los comentarios
for video_id in com_anuales['ID del Video']: # se recorren los IDs de los videos
    comments = get_video_comments(video_id) # se obtienen los comentarios del video
    video_comments_df = pd.DataFrame(comments) # se crea un DataFrame con los comentarios del video
    all_comments_df = pd.concat([all_comments_df, video_comments_df], ignore_index=True) # se concatenan los comentarios del video al DataFrame principal de comentarios

# Guardar el DataFrame en un archivo CSV
all_comments_df.to_csv('comentarios_yt', index=False) # se almacenan los comentarios en un archivo CSV

In [ ]:
all_comments_df

In [ ]:
import pandas as pd

In [ ]:
comentarios = pd.read_csv("./data/complete/comentarios_videos.csv")

In [ ]:
comentarios